In [1]:
#imports 

import numpy as np
import pandas as pd 
import seaborn as sns 
import openpyxl
import pickle 


In [2]:
df = pd.read_excel('C:\\Users\\Malavi\\Desktop\\RFISUBCODDES Predictor\\Ancillary Rev - 2015 - 2023.xlsx', header=2)
df

,Flight Date,Point of Sale,Channel,RFISUBCOD,RFISUBCODDES,RFICOD,REMARKS,Count,Sum of Revenue USD
0,2015-01-01,Sri Lanka,ATO/CTO/GSA/PSA,0BJ,UPGRADE,Air transportation,UPGRADE,2,1223.78
1,2015-01-01,Sri Lanka,Call centre,0BJ,UPGRADE,Air transportation,UPGRADE,61,13516.32
2,2015-01-01,Hong Kong,ATO/CTO/GSA/PSA,0IK,EXCESS WEIGHT,Baggage,EXCESS WEIGHT,4,384.23
3,2015-01-01,India Tamilnadu - Chennai,ATO/CTO/GSA/PSA,0IK,EXCESS WEIGHT,Baggage,EXCESS WEIGHT,26,1658.70
4,2015-01-01,India Trivandrum,ATO/CTO/GSA/PSA,0IK,EXCESS WEIGHT,Baggage,EXCESS WEIGHT,6,272.64
...,...,...,...,...,...,...,...,...,...
149997,2020-01-07,India Northern,IBE,0EO,CARBON OFFSET,Financial Impact,CARBON OFFSET,2,4.06
149998,2020-01-07,Singapore,IBE,0EO,CARBON OFFSET,Financial Impact,CARBON OFFSET,5,12.60
149999,2020-01-07,Sri Lanka,IBE,0EO,CARBON OFFSET,Financial Impact,CARBON OFFSET,1,3.04
150000,2020-01-07,United Kingdom,IBE,0EO,CARBON OFFSET,Financial Impact,CARBON OFFSET,5,25.08


In [3]:
# Convert Flight Date to datetime and extract Year & Month


df['Flight Date'] = pd.to_datetime(df['Flight Date'])
df['Year'] = df['Flight Date'].dt.year
df['Month'] = df['Flight Date'].dt.month

# Aggregate revenue by Year, Month, Point of Sale, RFISUBCODDES, and Count

df_main = df.groupby(['Year', 'Month', 'Point of Sale', 'RFISUBCODDES'])[['Sum of Revenue USD', 'Count']].sum().reset_index()

In [4]:
df_main

,Year,Month,Point of Sale,RFISUBCODDES,Sum of Revenue USD,Count
0,2015,1,Abu Dhabi & Al Ain,EXCESS WEIGHT,1952.19,46
1,2015,1,China South,EXCESS WEIGHT,6450.47,35
2,2015,1,China South,UPGRADE,201.07,1
3,2015,1,Dubai & Rest,EXCESS WEIGHT,53.12,1
4,2015,1,France,EXCESS WEIGHT,1401.73,18
...,...,...,...,...,...,...
11574,2020,1,United Kingdom,UPTO22LB/10KG - BAGGAGE,274.18,3
11575,2020,1,United States of America,EXCESS PIECE,832.96,13
11576,2020,1,United States of America,PRE PAID BAGGAGE,2942.79,36
11577,2020,1,United States of America,PRE-RESERVED SEAT ASSIGNMENT,3441.76,273


In [5]:
df_main.isnull().sum()


Year                  0
Month                 0
Point of Sale         0
RFISUBCODDES          0
Sum of Revenue USD    0
Count                 0
dtype: int64